In [ ]:
import numpy as np
import pylab
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from scipy import signal
from scipy import interpolate
from scipy.fftpack import fft
from scipy.signal import hilbert
from pycbc.waveform import get_td_waveform

In [ ]:
# define sampling frequency
fs = 4096
dt = 1/fs

In [ ]:
# load psd data
avg_psd_H1 = np.load("/Users/jkliao117/Desktop/MSci/avg psd H1.npz")['avg_psd_H1']
avg_psd_L1 = np.load("/Users/jkliao117/Desktop/MSci/avg psd L1.npz")['avg_psd_L1']
psdfreqs = np.load("/Users/jkliao117/Desktop/MSci/psd freqs.npz")['psdfreqs']

In [ ]:
# generate noise
def noise(data_size, f_high_pass = 35, f_low_pass = 350):
    # define sampled frequency points using the specified data size
    freqs = np.fft.rfftfreq(data_size, d=1/fs)
    # obtain complete power spectrum by interpolation of avergae psd
    interp_func = interpolate.interp1d(psdfreqs,avg_psd_H1)
    power = interp_func(freqs)
    # generate noise in frequency domian using  power spectrum
    fnoise = np.random.normal(0,np.sqrt(power/2.))+1.j*np.random.normal(0,np.sqrt(power/2.))
    # define bandpass frequencies
    mask = (freqs > f_high_pass) & (freqs < f_low_pass)
    # define window
    bandp = signal.tukey(np.sum(mask),alpha=0.25)
    # bandpass and window noise 
    fnoise_banded = np.zeros_like(fnoise)
    fnoise_banded[mask] = fnoise[mask]*bandp
    # inverse FFT back to time domain 
    tnoise = (fs**0.5)*np.fft.irfft(fnoise_banded, norm="ortho")
    return tnoise

In [ ]:
# generate GW signal
def wave(data_size, m1 =50, m2 = 50, d = 500):
    # generate gravitational wave using pycbc package
    hp, hc = get_td_waveform(approximant = 'SEOBNRv2',
                                mass1 = m1,
                                mass2 = m2,
                                delta_t = dt,
                                f_lower = 30, #Lowest frequency
                                distance = d # Luminosity Distance in Mpc
                            )
    h = np.asarray(hp)
    # find time duration of wave
    h_time = np.asarray(hp.sample_times)
    wave_size = h_time.shape[0]
    # hilbert transform
    analytic_signal = hilbert(h)
    # get amplitude, phase, and angular frequency
    amplitude_envelope = np.abs(analytic_signal)
    inst_phase = np.unwrap(np.angle(analytic_signal))
    inst_ang_frequency = np.diff(inst_phase)/(2.*np.pi)
    # randomise start time of GW event 
    start_time = int(np.random.uniform(0,data_size-wave_size))
    end_time = int(start_time+wave_size)
    
    GW_signal = np.zeros([data_size])
    # add gravitational wave
    GW_signal[start_time:end_time] = h
    # add cosine wave before gravitational wave using the amplitude, phase, and angular frequency found previously
    pre_h_time = np.arange(start_time)
    GW_signal[:start_time] = amplitude_envelope[102]*np.cos(inst_phase[102]-2*np.pi*inst_ang_frequency[102]*pre_h_time)
    return GW_signal, end_time

In [ ]:
# generate simulated data
def classify_dataset(num,data_size):
    sim_data_dataset= np.zeros((num,data_size))
    labels = np.zeros((num,2))
    for idx in range(num):
        a = np.random.uniform()
        noise_data = noise(data_size, f_high_pass = 35, f_low_pass = 350)
        if a > 0.5: 
            m1 = np.random.uniform(10,100)
            m2 = np.random.uniform(10,m1)
            d = np.random.uniform(100,1000)
            signal_data, end_time = wave(data_size, m1, m2, d)
            snr = np.sum(np.abs(signal_data[:end_time])**2)/np.sum(np.abs(noise_data[:end_time])**2)
            labels[idx] = 1, snr
            sim_data_dataset[idx] = noise_data + signal_data
        else:
            sim_data_dataset[idx] = noise_data
    return sim_data_dataset, labels

In [ ]:
# generate simulated data
def dataset(num,data_size):
    sim_data_dataset= np.zeros((num,data_size))
    labels = np.zeros((num,2))
    for idx in range(num):
        noise_data= noise(data_size, f_high_pass = 35, f_low_pass = 350)
        m1 = np.random.uniform(10,100)
        m2 = np.random.uniform(10,m1)
        d = np.random.uniform(100,1000)
        chirp_m = ((m1*m2)**0.6)/((m1+m2)**0.2)
        signal_data, end_time = wave(data_size, m1, m2, d)
        labels[idx] = chirp_m, d
        sim_data_dataset[idx] = noise_data + signal_data
    return sim_data_dataset, labels

In [ ]:
def norm(x):
    return (x-np.mean(x))/ np.std(x)